# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
from scipy import stats

Загрузим все необхдимые библиотеки и инструменты для работы.

In [2]:
warnings.filterwarnings("ignore")

Отключим предупреждения.

In [3]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Загрузим данные

In [4]:
data_0.head(5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_1.head(5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.head(5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [11]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [12]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [13]:
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

Столбец "уникальный идентификатор скважины" не влияет на решение поставленной задачи. Его можно удалить.

Мы загрузили все необходимые инструменты для работы, загрузили данные о трех скважинах. Изучили данные и подготовили их для обучения моделей.

## Обучение и проверка модели

In [14]:
data_0_train, data_0_valid = train_test_split(data_0, test_size=0.25, random_state=12345)
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25

In [15]:
features_0_train = data_0_train.drop(['product'], axis=1)
target_0_train = data_0_train['product']
features_0_valid = data_0_valid.drop(['product'], axis=1)
target_0_valid = data_0_valid['product']

features_1_train = data_1_train.drop(['product'], axis=1)
target_1_train = data_1_train['product']
features_1_valid = data_1_valid.drop(['product'], axis=1)
target_1_valid = data_1_valid['product']

features_2_train = data_2_train.drop(['product'], axis=1)
target_2_train = data_2_train['product']
features_2_valid = data_2_valid.drop(['product'], axis=1)
target_2_valid = data_2_valid['product']

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

In [16]:
print(features_0_train.shape)
print(features_0_valid.shape)
print(target_0_train.shape)
print(target_0_valid.shape)
print(features_1_train.shape)
print(features_1_valid.shape)
print(target_1_train.shape)
print(target_1_valid.shape)
print(features_2_train.shape)
print(features_2_valid.shape)
print(target_2_train.shape)
print(target_2_valid.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)
(75000, 3)
(25000, 3)
(75000,)
(25000,)
(75000, 3)
(25000, 3)
(75000,)
(25000,)


Проверим выборки.

In [17]:
model = LinearRegression()# инициализируем модель LinearRegression
model.fit(features_0_train, target_0_train) # обучим модель на тренировочной выборке
predictions_0_valid = model.predict(features_0_valid) # получим предсказания модели на валидационной выборке

Обучим модель и сделаем предсказания на валидационной выборке для региона 0.

In [18]:
model.fit(features_1_train, target_1_train) # обучим модель на тренировочной выборке
predictions_1_valid = model.predict(features_1_valid) # получим предсказания модели на валидационной выборке

Обучим модель и сделаем предсказания на валидационной выборке для региона 1.

In [19]:
model.fit(features_2_train, target_2_train) # обучим модель на тренировочной выборке
predictions_2_valid = model.predict(features_2_valid) # получим предсказания модели на валидационной выборке

Обучим модель и сделаем предсказания на валидационной выборке для региона 2.

In [20]:
print(predictions_0_valid.mean())
print(mean_squared_error(target_0_valid, predictions_0_valid)**0.5)

92.59256778438035
37.5794217150813


In [21]:
print(predictions_1_valid.mean())
print(mean_squared_error(target_1_valid, predictions_1_valid)**0.5)

68.728546895446
0.893099286775617


In [22]:
print(predictions_2_valid.mean())
print(mean_squared_error(target_2_valid, predictions_2_valid)**0.5)

94.96504596800489
40.02970873393434


Вывели на экран средний запас предсказанного сырья и значение RMSE

Мы обучили модели по всем трём месторождениям. Вывели на экран средний запас предсказанного сырья и значение RMSE. Больше всего запас сырья в скважине № 2. Немного меньше в скважине №0 и сильно уступает по этому параметру скважина № 1. 

## Подготовка к расчёту прибыли

In [23]:
BUDGET = 10000000000
PRICE = 450000
MONEY_FOR_POINT = BUDGET/200
print(MONEY_FOR_POINT)

50000000.0


Подсчитаем сколько денег выделяют на одну точку.

In [24]:
BARR=MONEY_FOR_POINT/PRICE
print(BARR)

111.11111111111111


Подсчитаем, сколько нужно тысяч баррелей для того, чтобы окупить затраты

In [25]:
print(BARR - target_0_valid.mean()) # Объём сырья для безубыточной разработки скважины № 0
print(BARR - target_1_valid.mean()) # Объём сырья для безубыточной разработки скважины № 1 
print(BARR - target_2_valid.mean()) # Объём сырья для безубыточной разработки скважины № 2 

19.032514370281845
42.38797508675114
16.226878302256736


Разработка скважины № 1 слишком затратна. Посмотрим,что получится при дальнейшем изучении.

## Расчёт прибыли и рисков 

In [26]:
def profit(target, predictions, count):
    profit=0
    probs_sorted = predictions.sort_values(ascending=False).head(200)
    selected = target[probs_sorted.index][:count] 
    return PRICE * selected.sum() - BUDGET

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [27]:
data_0_valid['product_pred']=predictions_0_valid

profit0=profit(data_0_valid['product'], data_0_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit0/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 3321 млн.


In [28]:
data_1_valid['product_pred']=predictions_1_valid

profit1=profit(data_1_valid['product'], data_1_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit1/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 2415 млн.


In [29]:
data_2_valid['product_pred']=predictions_2_valid

profit2=profit(data_2_valid['product'], data_2_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit2/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 2710 млн.


In [30]:
def boots (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    counter=0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        
        values.append(profit(target_subsample, preds_subsample, 500))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print('Вероятность убытков',stats.percentileofscore(values, 0),'%')
    print("Среднее значение бутстрепа", (values.mean())/1000000, "млн.")
    print("Верхняя граница доверительного интервала", higher)
    print("Нижняя граница доверительного интервала", lower)
    print()
print("Для первого месторождения")
boots(data_0_valid['product'], data_0_valid['product_pred'])
print()
print("Для второго месторождения")
boots(data_1_valid['product'], data_1_valid['product_pred'])
print()
print("Для третьего месторождения")
boots(data_2_valid['product'], data_2_valid['product_pred'])

Для первого месторождения
Вероятность убытков 2.0 %
Среднее значение бутстрепа 600.7352442611652 млн.
Верхняя граница доверительного интервала 1231163605.7914982
Нижняя граница доверительного интервала 12948331.135115242


Для второго месторождения
Вероятность убытков 0.30000000000000004 %
Среднее значение бутстрепа 665.2410582210723 млн.
Верхняя граница доверительного интервала 1197641587.4631522
Нижняя граница доверительного интервала 157988481.31991574


Для третьего месторождения
Вероятность убытков 3.0 %
Среднее значение бутстрепа 615.5597228409678 млн.
Верхняя граница доверительного интервала 1230644473.941316
Нижняя граница доверительного интервала -12218495.237293953



Вывод:<br>
    Большое кол-во скважин с лучшими запасами имеет регион № 1, который показал прибыль 3321 млн на лучших предсказанных данных;<br>
    Но выбирать регион только по лучшей прибыли некорректно, т.к. надо рассматривать и риски и расходы, особенно, когда на разработку нужно потратить 10 млрд рублей;<br>
    Основной критерий - вероятность убытков - 2.5 %. Регион № 2 имеет самую низкую вероятность убытков, всего 1,7%;<br>
    Остальные регионы имеют риск, превышающий основной критерий, поэтому рассмотрены быть не могут, хотя регион № 3 имеет более высокие значения верхней границы доверительного интервала.<br>
    Исходя из всего проведенного анализа, можно смело заявить, что второе месторождение самое выгодное, об этом говорит и показатель RMSE и процент убытков.<br>